In [7]:
import tkinter as tk
from tkinter import messagebox
import torch
from torchvision import transforms
from PIL import Image, ImageDraw
import numpy as np
from torchvision import datasets, transforms

# Define transformations for the training and test data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize the data to mean=0.1307 and std=0.3081
])

# Download and load the training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for training and test datasets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

print("Data loaders created successfully!")


Data loaders created successfully!


In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Convolutional layer 1: 1 input channel (grayscale), 32 output channels, 3x3 kernel size
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # Convolutional layer 2: 32 input channels, 64 output channels, 3x3 kernel size
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Max-pooling layer: 2x2 window size
        self.pool = nn.MaxPool2d(2, 2)
        # Fully connected layer 1: 64*7*7 input features, 128 output features
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        # Fully connected layer 2: 128 input features, 10 output features (for 10 classes)
        self.fc2 = nn.Linear(128, 10)
        # Dropout layer to avoid overfitting
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Apply ReLU activation, convolution, and pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Apply ReLU activation, convolution, and pooling
        x = x.view(-1, 64 * 7 * 7)  # Flatten the feature maps
        x = torch.relu(self.fc1(x))  # Apply ReLU activation to fully connected layer 1
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)  # Output layer
        return x

# Initialize the model
model = CNN()

print("Model created successfully!")


Model created successfully!


In [3]:
# Check if a GPU is available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the selected device (GPU or CPU)
model.to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with GPU support
num_epochs = 5  # You can increase it later

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Move data (images and labels) to the selected device
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

print("Training completed!")


Using device: cpu
Epoch [1/5], Loss: 0.1682
Epoch [2/5], Loss: 0.0567
Epoch [3/5], Loss: 0.0411
Epoch [4/5], Loss: 0.0317
Epoch [5/5], Loss: 0.0244
Training completed!


In [4]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the model on the 10000 test images: 99.29%


In [5]:
torch.save(model.state_dict(), 'mnist_cnn.pth')
print("Model saved successfully!")


Model saved successfully!


In [6]:
# Initialize a new instance of the model and load the saved weights
loaded_model = CNN()
loaded_model.load_state_dict(torch.load('mnist_cnn.pth'))

# Set the model to evaluation mode
loaded_model.eval()
print("Model loaded successfully!")


Model loaded successfully!


In [17]:
# Initialize Tkinter root window
root = tk.Tk()
root.title("MNIST Digit Drawer")
root.geometry("600x600")

# Canvas for drawing
canvas = tk.Canvas(root, width=500, height=500, bg="white")
canvas.pack(pady=20)

# Image for drawing, used for prediction
img = Image.new("L", (500, 500), 255)  # L mode for grayscale
draw = ImageDraw.Draw(img)


In [18]:
# Draw method for the canvas
def draw_on_canvas(event):
    x1, y1 = (event.x - 5), (event.y - 5)
    x2, y2 = (event.x + 5), (event.y + 5)
    canvas.create_oval(x1, y1, x2, y2, fill="black", width=10)
    draw.line([x1, y1, x2, y2], fill=0, width=10)

# Set up event bindings
canvas.bind("<B1-Motion>", draw_on_canvas)


'2165307168128draw_on_canvas'

In [19]:
# Function to clear the canvas
def clear_canvas():
    canvas.delete("all")
    draw.rectangle([0, 0, 500, 500], fill=255)


In [20]:
# Function to predict the digit
def predict_digit():
    # Convert canvas drawing to a format that the model understands
    img_resized = img.resize((28, 28), Image.Resampling.LANCZOS)  # Resize to 28x28 for MNIST
    img_tensor = transform(img_resized).unsqueeze(0)  # Add batch dimension

    # Get model prediction
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
    
    # Display the prediction result
    messagebox.showinfo("Prediction", f"Predicted digit: {predicted.item()}")


In [21]:
# Create buttons for prediction and clearing the canvas
predict_button = tk.Button(root, text="Predict", command=predict_digit, width=20, height=2)
predict_button.pack(side=tk.LEFT, padx=10)

clear_button = tk.Button(root, text="Clear", command=clear_canvas, width=20, height=2)
clear_button.pack(side=tk.RIGHT, padx=10)


In [22]:
# Run the application
root.mainloop()
